In [0]:
%%capture
!pip install category_encoders

In [2]:
import numpy as np #turning lists into arrays
import seaborn as sns #data loader
import pandas as pd #manipulate data
import matplotlib.pyplot as plt
import category_encoders as ce #encoding
from sklearn.model_selection import train_test_split # data splitting
from sklearn.preprocessing import MinMaxScaler # normalization
from sklearn.impute import SimpleImputer # fill NaN values
from sklearn.pipeline import make_pipeline # creating a pipeline
from sklearn.metrics import accuracy_score #scoring metric
from tensorflow import keras # for the neural network
from tensorflow.keras.models import Sequential # the model
from tensorflow.keras.layers import Dense, Dropout # layers for nerual network
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [31]:
url = "https://raw.githubusercontent.com/mpHarm88/DS-Unit-4-Sprint-2-Neural-Networks/master/module4-Hyperparameter-Tuning/WA_Fn-UseC_-Telco-Customer-Churn%2B(1).csv"
df= pd.read_csv(url)
print(df.shape)
df.head(3)

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [0]:
mapper = {"Yes": 1,
          "No": 0}

df["Churn"] = df["Churn"].map(mapper)

df["TotalCharges"] = df["TotalCharges"].str.strip()
df["TotalCharges"] = df["TotalCharges"].replace("", np.nan)
df["TotalCharges"] = df["TotalCharges"].replace(",", "")
df["TotalCharges"] = df["TotalCharges"].astype(float)

In [0]:
#pipeline for preprocessing
pipe = make_pipeline(
    ce.OneHotEncoder(), #encode categoricals
    SimpleImputer(strategy="median"), #fill the NaN's
    MinMaxScaler() # change values to be represented between 0 and 1
)

In [36]:
# drop uneeded columns
df = df.drop(["customerID"], axis=1)

# split features and targets
target = 'Churn'
X = df.drop(target, axis=1)
y = df[target]
print("Shape before split:")
print(f"X shape: {X.shape}\ny shape: {y.shape}")
print()

# split
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)
print("Shape after train test split:")
print(f"""X_train shape: {X_train.shape}
y_train shape: {y_train.shape}
X_test shape: {X_test.shape}
y_test shape: {y_test.shape}""")
print()

# apply pipe
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)
print("Shape after tranformation:")
print(f"""X_train Shape: {X_train.shape}
X_test Shape: {X_test.shape}
""")

# cast y as np.array for modeling
y_train = np.array(y_train)
y_test = np.array(y_test)

Shape before split:
X shape: (7043, 19)
y shape: (7043,)

Shape after train test split:
X_train shape: (5282, 19)
y_train shape: (5282,)
X_test shape: (1761, 19)
y_test shape: (1761,)

Shape after tranformation:
X_train Shape: (5282, 45)
X_test Shape: (1761, 45)



In [44]:
dims = X_train.shape[1]

model = Sequential()

model.add(Dense(32, input_dim=dims, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(90, activation='relu'))
model.add(Dense(45, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(90, activation='relu'))
model.add(Dense(9, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

model.fit(X_train, y_train, epochs=1000, verbose=1)

Epoch 1/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.5356 - accuracy: 0.7302
Epoch 2/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4699 - accuracy: 0.7635
Epoch 3/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4588 - accuracy: 0.7779
Epoch 4/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4537 - accuracy: 0.7855
Epoch 5/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4429 - accuracy: 0.7840
Epoch 6/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4419 - accuracy: 0.7919
Epoch 7/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4435 - accuracy: 0.7963
Epoch 8/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4302 - accuracy: 0.7952
Epoch 9/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.4343 - accuracy: 0.7952
Epoch 10/1000
166/166 [==============================] - 0s 2ms/step - loss: 0.428

In [45]:
model.evaluate(X_test, y_test)

56/56 [==============================] - 0s 1ms/step - loss: 0.6930 - accuracy: 0.7609


[0.6929749250411987, 0.7609313130378723]